In [1]:
from Wavefronts import *
from Storage import *

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# More Efficient Algorithm

Intiate Algorithm 

In [2]:
Storage_Voltage_Active.append(Wavefront_Source(Voltage_Souce_Magnitude,0,Voltage_Source_Period_On,0))
Storage_Voltage_Active[0].about()


Some Information about a wavefront
Type :                             <class 'Wavefronts.Wavefront_Source'>
Poisiton Start :                   0
Poisiton End :                     0
Time Start :                       0
Time End :                         0.000012
Voltage Magnitude :                63
Current Magnitude :                0
Excitation Type :                  HIGH
Excitation Number :                0
Order :                            0
Inductive Reflections :            0
Capacitive Reflections :           0
Current Chain Reflections :        0
Transmission Ind -> Cap :          0
Transmission Cap -> Ind :          0


In [3]:
Storage_Merged : Wavefront = deque()
Storage_New : Wavefront = deque()
Storage_Temp_Away : Wavefront = deque()

In [4]:
temp_wavefront = Storage_Voltage_Active.popleft()
temp_wavefront.Generate(Storage_Temp_Away,Storage_Temp_Away)
Storage_Voltage_Completed.append(temp_wavefront)

In [5]:
is_Inductive = True

while len(Storage_Temp_Away) > 0:
    temp_wavefront = Storage_Temp_Away.popleft()
    temp_wavefront.Generate(Storage_Merged,Storage_Merged)

    if(is_Inductive):
        Storage_Inductor_Completed.append(temp_wavefront)
    else:
        Storage_Capacitor_Completed.append(temp_wavefront)

    is_Inductive = not is_Inductive

## Start of Main Algorithm

Populate NEW storage and store MERGED in COMPLETED

In [6]:
is_Inductive = True

while len(Storage_Merged) > 0:

    temp_wavefront = Storage_Merged.popleft()

    # Store wave from MERGED
    if(is_Inductive):
        Storage_Inductor_Completed.append(temp_wavefront)
    else:
        Storage_Capacitor_Completed.append(temp_wavefront)

    is_Inductive = not is_Inductive

    # Generate and Store Away Waves
    temp_wavefront.Generate(Storage_Temp_Away,Storage_Temp_Away)

    temp_inductive_wavefront = Storage_Temp_Away.popleft()
    temp_inductive_wavefront.Generate(Storage_New,Storage_New)
    Storage_Inductor_Completed.append(temp_inductive_wavefront)

    temp_capacitve_wavefront = Storage_Temp_Away.popleft()
    temp_capacitve_wavefront.Generate(Storage_New,Storage_New)
    Storage_Capacitor_Completed.append(temp_capacitve_wavefront)



In [7]:
is_Simulation_Completed = False
temp_first_wavefront = Storage_New.popleft()

if(Inductor_Time <= Capacitor_Time):
    if(temp_first_wavefront.time_start >= Simulation_Stop_Time):
        is_Simulation_Completed = True

Storage_Merged.append(temp_first_wavefront)

while len(Storage_New) > 0 :
    
    temp_wavefront = Storage_New.popleft()

    if len(Storage_New) == 0:
        if(Capacitor_Time <= Inductor_Time):
            is_Simulation_Completed = True
    else:
        temp_next_wavefront = Storage_New.popleft()

        temp_wavefront_inductive,temp_wavefront_capacitve = temp_wavefront.Generate_Return()
        temp_next_wavefront_inductive,temp_next_wavefront_capacitve = temp_next_wavefront.Generate_Return()

        temp_wavefront_inductive.Merge(temp_next_wavefront_inductive)
        temp_wavefront_capacitve.Merge(temp_next_wavefront_capacitve)

        

